### 一. 什么是数据倾斜, 如何处理
* **什么是数据倾斜**  
大量相同的key被partition到一台机器, 造成这台机器计算任务量极大, 而其他机器计算量任务极小  
* **即是数据倾斜, UDAF函数也可能不产生效率低下的问题?**  
像sum, max, min, count等UDAF不怕数据倾斜, 因为会在map端进行一次数据聚合  
* **count(distinct)最受数据倾斜影响?**  
因为count(distinct)要先按照group by分组, 再按distinct排序. 一般这中语句是很倾斜的;  
比如男uv，女uv，淘宝一天30亿的pv，如果按性别分组，分配2个reduce，每个reduce处理15亿数据。   
* **如何处理**
    1. key上加上随机数打散, 然后最后计算结果时恢复成原来的key
    2. 过滤空值, 或者将空值变成一个字符串并加上随机前缀, 由于随机的key找不到相同的而执行失败
    3. 将倾斜的key拿出来单独处理, 最后和其它结果union all在一起(不去重不排序)
    4. 开启参数`hive.groupby.skewindata=true`  
       这会生成2个任务:   
        1. 第一个任务: map的输出结果随机的分配到reduce上, 每个reduce按照key做第一次聚合
        2. 第二个任务: 将相同的key分不到同一个partition上, 再做第二次聚合  
    5. 将`count(distinct)`替换成`sum() group by`
    
#### 2. 